In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Replace the datatypes of not float
# As tf does not allow dataframes (instead numpy arrays)
dataset["sex"].replace({"male": 0, "female": 1}, inplace = True)
dataset["smoker"].replace({"no": 0, "yes": 1}, inplace = True)
dataset["region"].replace({
    "northwest": 0,
    "northeast": 1,
    "southwest": 2,
    "southeast": 3
}, inplace = True)

dataset

In [ ]:
# Pop off the expenses column
dataset_y = dataset.pop('expenses')
dataset_y.head()

In [ ]:
# Split the dataframe into train and test
train_dataset, test_dataset, train_labels, test_labels = train_test_split(dataset, dataset_y, test_size = 0.2, random_state=42)

# Convert the pandas df to np arrays for tf to work
train_dataset = train_dataset.to_numpy().astype(np.float32)
test_dataset = test_dataset.to_numpy().astype(np.float32)
train_labels = train_labels.to_numpy().astype(np.float32)
test_labels = test_labels.to_numpy().astype(np.float32)

train_labels

In [ ]:
# Normalize input features first
normalizer = tf.keras.layers.Normalization(input_shape = [6,], axis = -1)  # 6 features
normalizer.adapt(train_dataset)

# Create tf model
model = tf.keras.Sequential([
    normalizer,
    tf.keras.layers.Dense(units = 32, activation = "relu"),
    tf.keras.layers.Dense(units = 1)
])

model.summary()

In [ ]:
# Compile model
model.compile(
    # Learning rate = 0.01 is important here -- it helps the model learns faster (than defaut 0.001)
    optimizer = tf.optimizers.Adam(learning_rate = 0.01),
    loss = "mae",
    metrics = ["mae", "mse"]
)

In [ ]:
# Fit model
history = model.fit(
    x = train_dataset,
    y = train_labels,
    epochs = 100,
)

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
